In [57]:
import requests
import telethon
from telethon import TelegramClient
import asyncio
import json

In [ ]:
api_id = 'api id от приложения тг'
api_hash = 'api hash от приложения тг'
phone = 'Телефон'

In [59]:
client = TelegramClient(phone, api_id, api_hash)

Парсинг группы

In [69]:
async def quick_chat_check(chat_id):
    async with TelegramClient('session_name', api_id, api_hash) as client:
        try:
            chat = await client.get_entity(int(chat_id))
            
            print("=== ИНФОРМАЦИЯ О ЧАТЕ ===")
            print(f"💬 Название: {getattr(chat, 'title', 'Без названия')}")
            print(f"🆔 ID: {chat.id}")
            print(f"👤 Username: @{getattr(chat, 'username', 'Нет')}")
            print(f"📊 Тип чата: {type(chat).__name__}")
            print(f"👥 Участников: {getattr(chat, 'participants_count', 'Неизвестно')}")
            
            # Быстрая проверка последних сообщений
            print(f"\n=== ПОСЛЕДНИЕ 10 СООБЩЕНИЙ ===")
            messages = await client.get_messages(chat, limit=100)
            
            for i, msg in enumerate(messages, 1):
                sender = await msg.get_sender()
                sender_name = getattr(sender, 'first_name', getattr(sender, 'title', 'Неизвестно'))
                message_preview = msg.text[:60] + '...' if msg.text and len(msg.text) > 60 else (msg.text or '[медиа]')
                print(f"{i:2d}. {sender_name}: {message_preview}")
            
        except Exception as e:
            print(f"❌ Ошибка: {e}")

In [70]:
await quick_chat_check('-4512620959')

=== ИНФОРМАЦИЯ О ЧАТЕ ===
💬 Название: Чисто КэСэДва и ф0рест
🆔 ID: 4512620959
👤 Username: @Нет
📊 Тип чата: Chat
👥 Участников: 7

=== ПОСЛЕДНИЕ 10 СООБЩЕНИЙ ===
 1. Ruslan: хахахахаха
 2. Брюхич🕸: 😌 **ПРОСТРЕЛ. В СМОК. БЕЗ ЗУМА. В ГОЛОВУ. ОТОМСТИЛ.**
 3. Брюхич🕸: Бля че сегодня происходит
 4. Ruslan: Это я когда карты пикаем
 5. Брюхич🕸: 🇮🇹😁** tN1R решил применить **[**секретную стратегию**](https...
 6. Брюхич🕸: 🤣
 7. Вовчик🕸️: а че  спириты 1:9 летят
 8. Ruslan: Нихуя всадил
 9. Брюхич🕸: 🙂 Как это зарегало? Тот самый VAC от игрока Inner Circle.
10. Ruslan: Дефолтный тиммейт Руслана в катке на повышение:
11. Ruslan: мне кажется так и есть
12. Ruslan: хорош
13. Брюхич🕸: ёбаные 5лвла
14. Брюхич🕸: 1 в 1 выходили
15. Брюхич🕸: Че в голове бля должно быть
16. yungyuga 🕸️: просто типы лс ебанные
17. Брюхич🕸: Ч за х бля
18. Брюхич🕸: Хотя инста работает
19. Брюхич🕸: [медиа]
20. Брюхич🕸: Они че со смоками  сделали
21. Брюхич🕸: И такая хуйня не только на мираже
22. Брюхич🕸: Бля 3-4к ело типы смок

In [72]:
await quick_chat_check('@exploitex')

❌ Ошибка: invalid literal for int() with base 10: '@exploitex'



Парсинг канала, где больше 200 подписчиков


In [73]:
async def parse_channel(channel_username_or_id):
    async with TelegramClient('session_name', api_id, api_hash) as client:
        await client.start(phone=phone)  # или phone=phone_number
        
        print(f"🔍 Парсинг канала: {channel_username_or_id}")
        
        try:
            # Получаем канал
            channel = await client.get_entity(channel_username_or_id)
            
            print("✅ КАНАЛ НАЙДЕН:")
            print(f"   📢 Название: {getattr(channel, 'title', 'Без названия')}")
            print(f"   🔢 ID: {channel.id}")
            print(f"   👤 Username: @{getattr(channel, 'username', 'Нет username')}")
            print(f"   📝 Описание: {getattr(channel, 'about', 'Нет описания')}")
            print(f"   👥 Подписчиков: {getattr(channel, 'participants_count', 'Неизвестно')}")
            
            # Получаем сообщения
            print(f"\n📨 Получаем сообщения...")
            messages_data = []
            
            async for message in client.iter_messages(channel, limit=100):
                message_info = {
                    'id': message.id,
                    'date': message.date.isoformat(),
                    'text': message.text or '',
                    'views': getattr(message, 'views', 0),
                    'forwards': getattr(message, 'forwards', 0),
                    'media': bool(message.media),
                    'media_type': str(message.media.__class__.__name__) if message.media else None
                }
                messages_data.append(message_info)
                
                # Выводим в консоль
                views = getattr(message, 'views', 0)
                print(f"   [{message.date.strftime('%d.%m %H:%M')}] 👁️ {views} | {message.text[:80]}{'...' if len(message.text or '') > 80 else ''}")
            
            # Сохраняем в JSON
            channel_data = {
                'channel_info': {
                    'name': getattr(channel, 'title', 'Без названия'),
                    'id': channel.id,
                    'username': getattr(channel, 'username', ''),
                    'description': getattr(channel, 'about', ''),
                    'subscribers': getattr(channel, 'participants_count', 0)
                },
                'messages': messages_data
            }
            
            filename = f"channel_{getattr(channel, 'username', channel.id)}.json"
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(channel_data, f, ensure_ascii=False, indent=2)
            
            print(f"\n🎉 ПАРСИНГ ЗАВЕРШЕН!")
            print(f"   💾 Сохранено сообщений: {len(messages_data)}")
            print(f"   📁 Файл: {filename}")
            
            return channel_data
            
        except Exception as e:
            print(f"❌ Ошибка: {e}")
            return None

  # или по ID: -100123456789

In [77]:
# Запуск для канала по username
await parse_channel('exploitex')

🔍 Парсинг канала: exploitex
✅ КАНАЛ НАЙДЕН:
   📢 Название: Эксплойт
   🔢 ID: 1307778786
   👤 Username: @exploitex
   📝 Описание: Нет описания
   👥 Подписчиков: None

📨 Получаем сообщения...
   [15.10 14:02] 👁️ 40492 | ⚡️ Доллар упал до 76 рублей.

Очень вовремя.

@exploitex
   [15.10 13:37] 👁️ 65676 | **Новый iPad Pro на M5 **— это буквально MacBook в планшете.

Толщина всего 5,1 ...
   [15.10 13:13] 👁️ 82056 | ⚡️ **MacBook Pro на M5 в 3,5 раза (!)** [быстрее](https://www.apple.com/macbook-...
   [15.10 13:08] 👁️ 91496 | 
   [15.10 13:08] 👁️ 94703 | **⚡️ MacBook Pro M5 и iPad Pro анонсированы —** Apple [открыла](https://www.appl...
   [15.10 12:30] 👁️ 91790 | 
   [15.10 12:30] 👁️ 91539 | 
   [15.10 12:30] 👁️ 92105 | 
   [15.10 12:30] 👁️ 88843 | 
   [15.10 12:30] 👁️ 90975 | 
   [15.10 12:30] 👁️ 88736 | 
   [15.10 12:30] 👁️ 88549 | **Плюшевые Пепе нашли новый дом —** подписчики присылают фото наших игрушек по м...
   [15.10 10:57] 👁️ 133700 | 
   [15.10 10:57] 👁️ 135553 | 
   [15.10 10:5

{'channel_info': {'name': 'Эксплойт',
  'id': 1307778786,
  'username': 'exploitex',
  'description': '',
  'subscribers': None},
 'messages': [{'id': 28677,
   'date': '2025-10-15T14:02:38+00:00',
   'text': '⚡️ Доллар упал до 76 рублей.\n\nОчень вовремя.\n\n@exploitex',
   'views': 40492,
   'forwards': 1531,
   'media': True,
   'media_type': 'MessageMediaPhoto'},
  {'id': 28676,
   'date': '2025-10-15T13:37:39+00:00',
   'text': '**Новый iPad Pro на M5 **— это буквально MacBook в планшете.\n\nТолщина всего 5,1 мм — тоньше, чем айфон. В основе мощь полноценного процессора M5: LLM и генеративные модели **можно запускать прямо на устройстве.** \n\nТакже внутри до 16 ГБ оперативки, до 2 ТБ SSD, Ultra Retina XDR дисплей до 13 дюймов с 1600 нит. \n\nЦены стартуют от $999 (79 тыс. рублей) за 11 дюймов и $1299 (103 тыс. рублей) за 13 дюймов. Продажи начинаются 22 октября.\n\n@exploitex',
   'views': 65676,
   'forwards': 637,
   'media': True,
   'media_type': 'MessageMediaPhoto'},
  {'id'